In [1]:
import pandas as pd
from myfun.addcol import addcolumns
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
import joblib
import os

#### 同地同時法之訓練程式碼(只使用中央氣象局每隔10分鐘的第三方資料進行訓練)

In [ ]:
df1 = pd.read_csv(f"./train_combine_groupby/01_traingroupby.csv")
df1 = addcolumns(df1)
def combine_weather(df1:pd.DataFrame):   ###定義函數與中央氣象局每隔10分鐘的天氣資訊合併
    weather_df = pd.read_csv("../資料前處理以及第三方資料/10mins_weather_data.csv")  
    weather_df["DateTime"] =  pd.to_datetime(weather_df["DateTime"])
    weather_df["日期"] = pd.to_datetime(weather_df["DateTime"].dt.date)
    weather_df["時間"] = weather_df["DateTime"].dt.hour.astype(str).str.zfill(2)
    weather_df["時段"] = weather_df["DateTime"].dt.minute.astype(str).str.zfill(2)
    
    weather_df.fillna(0,inplace=True)
    df = pd.merge(
        df1,
        weather_df,
        left_on=["日期","時間","時段"],
        right_on=["日期","時間","時段"],
        how="left",
        suffixes=["","_r"]
    )

    return(df)
comdf = combine_weather(df1)

In [ ]:
def selecttime(df:pd.DataFrame):  #挑選訓練集的時間  可自行定義調整
    df["TimeGroup"] = pd.to_datetime(df["TimeGroup"])
    df = df[(df["TimeGroup"].dt.hour >= 9) & (df["TimeGroup"].dt.hour < 17)| ((df["TimeGroup"].dt.hour == 17) & (df["TimeGroup"].dt.minute == 0))]
    return(df)
selecttime(combine_weather(df1)).head(2)

,Serial,LocationCode,TimeGroup,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),日期,...,SunshineDuration,UVIndex,AirTemperature,DailyExtremeHighAirTemperature,DailyExtremeLowAirTemperature,RelativeHumidity,VisibilityDescription,StationLatitude,StationLongitude,StationAltitude
15,20240101090001,1,2024-01-01 09:00:00,1.91588,1017.906,18.33,92.29,8395.250,17.368,2024-01-01,...,0.0,0.0,18.8,19.3,18.6,64.0,7-10,23.975128,121.61327,16.1
16,20240101091001,1,2024-01-01 09:10:00,0.09000,1017.965,18.02,97.37,7046.501,12.612,2024-01-01,...,0.0,0.0,18.9,19.3,18.6,64.0,21-30,23.975128,121.61327,16.1


In [5]:
os.cpu_count()  #查看有多少運算資源可用

16

In [ ]:
d = {}
for i in range(1,18):
    l1 = "0" + str(i) if i<10 else str(i)
    df1 = pd.read_csv(f"./train_combine_groupby/{l1}_traingroupby.csv")
    df1 = addcolumns(df1)
    df1 = combine_weather(df1)
    df1 = selecttime(df1)
    df1.dropna(inplace=True)
    # 以中央氣象局每隔10分鐘的天氣資料當作訓練集X  訓練集Y則為對應到時間的發電量
    X = df1.loc[:,[ 'SunshineDuration', 'UVIndex',
                'AirTemperature', 'DailyExtremeHighAirTemperature',
                'DailyExtremeLowAirTemperature', 'RelativeHumidity',
                ]].reset_index(drop=True)
    
    Y = df1.loc[:,["Power(mW)"]].reset_index(drop=True)
    Y = Y.values.ravel()
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.01, random_state=42) #採用交叉驗證  拿全部資料來做訓練   測試集僅需要少量資料供參考

    model = RandomForestRegressor(
        n_jobs=12
    )  ## 可自行調整參數用 n_estimators= , max_depth=, min_samples_split=
    scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=5)
    MAE = -sum(scores)
    model.fit(X_train, y_train)
    
    model_file_name = f'{l1}_weather.pkl'  #權重命名為裝置名稱_weather.pkl
    model_folder_name = "repeat_train_use_full_data_update_weather_datasingle_location"         #將模型相關資訊儲存在這個資料夾下(在競賽結束後有重新測試過本程式碼因此在前面加上repeat_)
    model_file_path = os.path.join("./model_each_location",model_folder_name,model_file_name)
    os.makedirs(os.path.dirname(model_file_path), exist_ok=True)
    joblib.dump(model, model_file_path)
    print(model_file_name ," MAE:",MAE )
    print(f"模型已保存至: {model_file_path}")  #紀錄每個裝置的模型MAE
    if model_file_name not in d:
        d[model_file_name] = MAE

01_weather.pkl  MAE: 593.9418815001604
模型已保存至: ./model_each_location\repeat_train_use_full_data_update_weather_datasingle_location\01_weather.pkl
02_weather.pkl  MAE: 691.5330873848827
模型已保存至: ./model_each_location\repeat_train_use_full_data_update_weather_datasingle_location\02_weather.pkl
03_weather.pkl  MAE: 752.4155791895244
模型已保存至: ./model_each_location\repeat_train_use_full_data_update_weather_datasingle_location\03_weather.pkl
04_weather.pkl  MAE: 626.2070831931559
模型已保存至: ./model_each_location\repeat_train_use_full_data_update_weather_datasingle_location\04_weather.pkl
05_weather.pkl  MAE: 776.3700988279736
模型已保存至: ./model_each_location\repeat_train_use_full_data_update_weather_datasingle_location\05_weather.pkl
06_weather.pkl  MAE: 859.1185987680649
模型已保存至: ./model_each_location\repeat_train_use_full_data_update_weather_datasingle_location\06_weather.pkl
07_weather.pkl  MAE: 822.2066641515225
模型已保存至: ./model_each_location\repeat_train_use_full_data_update_weather_datasingle_lo

# 紀錄每個model的MAE並且存成json檔案

In [8]:
import json
json_output = json.dumps(d,indent=4)
output_path = f"./model_each_location/{model_folder_name}/d.json"
with open(output_path, "w", encoding="utf-8") as file:
    file.write(json_output)